# pyscr/load_to_postgres.py

Load cleaned Instacart CSVs into PostgreSQL tables.


In [15]:

import os
import pandas as pd
from sqlalchemy import create_engine


In [ ]:

# -----------------------------
# 1. Database connection config
# -----------------------------
DB_USER = "postgres"
DB_PASS = "1998"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "instacart"
DB_SCHEMA = "instacart"

DB_URL = f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

engine = create_engine(DB_URL)

# Base folder for cleaned CSVs
DATA_CLEAN_PATH = r'data_clean'


In [17]:


def load_csv(table_name: str, filename: str, dtype=None):
    """
    Helper to read a cleaned CSV and load it into a Postgres table.
    """
    path = os.path.join(DATA_CLEAN_PATH, filename)
    print(f"\nLoading {filename} into {DB_SCHEMA}.{table_name} ...")
    df = pd.read_csv(path, dtype=dtype)

    print(df.head())

    df.to_sql(
        name=table_name,
        con=engine,
        schema=DB_SCHEMA,
        if_exists="append",   
        index=False,
        chunksize=100_000,
        method="multi",
    )
    print(f"Loaded {len(df):,} rows into {DB_SCHEMA}.{table_name}")


def main():
    load_csv(
        table_name="departments",
        filename="departments.csv",
        dtype={
            "department_id": "int64",
            "department": "string",
        },
    )

    load_csv(
        table_name="aisles",
        filename="aisles.csv",
        dtype={
            "aisle_id": "int64",
            "aisle": "string",
        },
    )

    load_csv(
        table_name="products",
        filename="products.csv",
        dtype={
            "product_id": "int64",
            "product_name": "string",
            "aisle_id": "int64",
            "department_id": "int64",
        },
    )

    load_csv(
        table_name="orders",
        filename="orders.csv",
        dtype={
            "order_id": "int64",
            "user_id": "int64",
            "eval_set": "string",
            "order_number": "int64",
            "order_dow": "int64",
            "order_hour_of_day": "int64",
            "days_since_prior_order": "float64",
        },
    )


    load_csv(
        table_name="order_products",
        filename="order_products.csv",
        dtype={
            "order_id": "int64",
            "product_id": "int64",
            "add_to_cart_order": "int64",
            "reordered": "int64",   
        },
    )


if __name__ == "__main__":
    main()



Loading departments.csv into instacart.departments ...
   department_id department
0              1     frozen
1              2      other
2              3     bakery
3              4    produce
4              5    alcohol
Loaded 21 rows into instacart.departments

Loading aisles.csv into instacart.aisles ...
   aisle_id                       aisle
0         1       prepared soups salads
1         2           specialty cheeses
2         3         energy granola bars
3         4               instant foods
4         5  marinades meat preparation
Loaded 134 rows into instacart.aisles

Loading products.csv into instacart.products ...
   product_id                                       product_name  aisle_id  \
0           1                         Chocolate Sandwich Cookies        61   
1           2                                   All-Seasons Salt       104   
2           3               Robust Golden Unsweetened Oolong Tea        94   
3           4  Smart Ones Classic Favorites Mini